In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
#    for filename in filenames:
#        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from fastai.vision.all import *

In [ ]:
df = pd.read_csv('../input/plant-pathology-2021-fgvc8/train.csv')
df.head()

In [ ]:
path = '../input/plant-pathology-2021-fgvc8'

def get_x(r):
    return path + '/train_images/' + r['image']

def get_y(r):
    return r['labels'].split(' ')

dblock = DataBlock(blocks=(ImageBlock, MultiCategoryBlock),
                   get_x = get_x, 
                   get_y = get_y,
                  item_tfms = RandomResizedCrop(128, min_scale = 0.35),
                  batch_tfms=aug_transforms())

In [ ]:
dls = dblock.dataloaders(df)
test_files = get_image_files(path + '/test_images')
test_files.sort()
test_dl = dls.test_dl(test_files)

dls.show_batch(nrows=1, ncols=3)

In [ ]:
f1score_multi = F1ScoreMulti()
model = xresnet50(n_out=dls.c)
learn = Learner(dls, model, metrics=f1score_multi)

In [ ]:
learn.lr_find()

In [ ]:
learn.fit_one_cycle(7, 3e-3)

In [ ]:
learn.show_results()

In [ ]:
tst_preds, tst_targs = learn.get_preds(dl=test_dl)
thresh = 0.5
labelled_preds = [' '.join([dls.vocab[i] for i,p in enumerate(pred) if p > thresh]) for pred in tst_preds]

In [ ]:
fnames = [(f.name[:-4] + '.jpg') for f in test_files]
final_df = pd.DataFrame({'image':fnames, 'labels':labelled_preds}, columns=['image', 'labels'])
final_df.to_csv('./submission.csv', index=False)

Adapted from: https://github.com/fastai/fastbook/blob/master/06_multicat.ipynb